入門ウェブハンドリングp176

$$
\sigma_r
$$

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from numba import njit

In [19]:
"""パラメータ"""
r_c = 50  # コア半径 [mm]
r_cin = 40  # コア内径 [mm]
e_cin = 1  # コアヤング率
nu_c = 1

N = 30000  # 巻数
nu = 1  # 半径方向ポアソン比
#T = 0.002  # シート厚み[mm]

# ポリプロピレン
K1 = 0.1e-3  # [GPa]
K2 = 260.0e-3  # [GPa]
dr = 25.4e-3 # ウェブ厚み[mm]
E_THETA = 1.65  # 接線方向ヤング率[GPa]

E_C = e_cin / (((r_c/r_cin)**2+1) / ((r_c/r_cin)**2-1) - nu_c)


T = 100 # 一定巻取り張力[N]

r = np.linspace(r_c, r_c+T*N, N)  # 半径リスト

In [23]:
#@njit

def Ai(r):
    return 1 + (3-nu)*dr / (2*r)

def Bi(r, g):
    return (1 + nu - g**2) * (dr/r)**2 - 2

def Ci(r):
    return 1 - (3-nu)/2*dr/r


def main():
    sigma_r = np.zeros(N)
    a = np.zeros(N)
    b = np.zeros(N)
    c = np.zeros(N)
    g = np.zeros(N)
    sigma_r_total = 0
    
    a = 1 + (3-nu) * dr / (2 * r)
    c = 1 - (3-nu)/2 * dr/r
    
    for n in range(N):  #巻数ループ
        if n == 0:
            sigma_r[0] = -T / (r_c + 1*dr)
        else:
            for i in range(n):
                if i == 0:
                    A1 = 1
                    B1 = (1 - E_THETA/E_C - nu)*dr/r_c - 1
                    sigma_r[1] = -T / (r_c + i*dr)
                    sigma_r[0] = A1 / B1 * -sigma_r[1]
                else:
                    # 係数g計算
                    g = np.zeros(N)  # g初期化
                    sigma_r_total = 0
                    for j in range(i):
                        sigma_r_total += sigma_r[j]
                        E_r = K2 * (sigma_r_total + K1)
                        g[j] = np.sqrt(E_THETA / E_r)
                    
                    A = np.zeros((i, i))
                    b = np.zeros((i, 1))
                    for j in range(i):
                        A[j, j-1] = Ai(r[j])
                        A[j, j] = Bi(r[j], g[j])
                        A[j, j+1] = Ci(r[j])